# Binary Star Simulation

This is the first attempt to build a full on binary star simulator for use in our introductory astronomy labs.

This code takes an approach to modelling binary stars outlined by Carroll and Ostlie in their *TwoStars* code, but it is optimized for Python extensively.

In [ ]:
# Developed by Juan Cabanela starting June 19, 2018 
#
# This simulation is meant to allow students to discover how adjusting the parameters of a model of a stellar system
# can allow use to model either radial velocity curves or light curves.
#
# This code started life as an extension of the Center of Mass interactive (by Sam Holen) but required the development
# of a Python implementation of a binary star model (based on approach used in the TwoStars code from
# Carrolll and Ostlie).

In [ ]:
## UNcomment to turn on autoreloading with each execution
#%load_ext autoreload
#%autoreload 2

from IPython.display import display, HTML
import numpy as np
import ipywidgets as widgets
import pythreejs as p3j
import bqplot as bq
import tempNcolor as tc
import starlib as star
import number_formatting as nf

In [ ]:
## FUNCTIONS ##

def ConfigBothStars(mass1, mass2):
    '''
    Determines the radii (in solar radii), temperature (in K), and hexcolor of the two stars assuming 
    they are main sequence stars and returns that information.  Does this by calling the ConfigStar 
    function for both stars.
    '''
    (radius1, temp1, hexcolor1) = star.ConfigStar(mass1)
    (radius2, temp2, hexcolor2) = star.ConfigStar(mass2)
    
    return (radius1, temp1, hexcolor1, radius2, temp2, hexcolor2)


def property_update(change=None):
    '''
    This function updates the stellar properties (and orbital properties) and is meant to be used
    when there are changes to stellar properties as controled by various ipywidgets on screen.
    '''
    global star1, star2, init_r1, init_r2, P_days, ap, aa, orbit_info, radvel_info, light_curve, collision
    global xmax, grid_step, surfgrid, orbit1_line, orbit2_line, N, multiplier, view_factor
        
    # determine parameters of the two stars
    (radius1, temp1, hexcolor1, radius2, temp2, hexcolor2) = ConfigBothStars(star1_slider.value, star2_slider.value)
    
    # Set luminosities
    L1_output.value = str(nf.SigFig((temp1/star.Te_Sun)**4 * radius1**2, 3))
    L2_output.value = str(nf.SigFig((temp2/star.Te_Sun)**4 * radius2**2, 3))

    # Recompute orbital information
    (P_days, ap, aa, maxrad, orbit_info, collision) = OrbitalInfo_RSun(star1_slider.value, star2_slider.value,
                                                                       radius1, radius2,
                                                                       semimajor_slider.value,
                                                                       ecc_slider.value,
                                                                       phi_slider.value, N)

    # If there is a collision, deal with it, otherwise do the rest of the work
    if (collision):
        phase_title.value = phase_title_collision
        phase_play.step = 0
        phase_play.disabled = True
        phase_slider.disabled = True
    else:
        # Make sure orbital phase can be adjusted
        phase_title.value = phase_title_default
        phase_play.step = 1
        phase_play.disabled = False
        phase_slider.disabled = False

    # Redraw orbits
    (orbit1_new, orbit2_new) = draw_orbits_RSun(orbit_info)
    orbit1_line.geometry = orbit1_new.geometry
    orbit2_line.geometry = orbit2_new.geometry
    
    # Redraw stars
    star1.position = [orbit_info['x1_RSun'][t_idx], orbit_info['y1_RSun'][t_idx], 0]
    star2.position = [orbit_info['x2_RSun'][t_idx], orbit_info['y2_RSun'][t_idx], 0]

    # Revise orbital readouts
    P_output.value = "{0:.1f}".format(P_days)
    ap_output.value = "{0:.1f}".format(ap)
    aa_output.value = "{0:.1f}".format(aa)

    # Update surface grid and axes
    (xmax, grid_step) = grid_setup(maxrad + max(radius1,radius2))
    gridsep_output.value = "{0:.2f}".format(grid_step)
    surf_new, surfgrid_new = star.xyplane(xmax, grid_step)  # Generate flat surface and grid for perspective
    surf.geometry = surf_new.geometry
    surfgrid.children = surfgrid_new.children
    
    # Adjust viewer region
    camera_update()

    # Define initial viewing position (adjusted for inclination)
    starcam.position = (view_factor*xmax*np.sin(incl_slider.value*star.deg2rad), 
                        0, 
                        view_factor*xmax*np.cos(incl_slider.value*star.deg2rad))
    # Tell the OrbitControl to update since I changed the position manually...
    controller.exec_three_obj_method('update',)
    
    # updates the radii and color of each star, it also rescales the 
    # multiplier so that stars are rendered larger-than-scale if view area becomes too large
    minratio = 50
    if (minratio*min(radius1, radius2) < xmax):
        multiplier = xmax/(minratio*min(radius1, radius2))
        sys_title.value = sys_title_scaling
    else:
        multiplier = 1
        sys_title.value = sys_title_default
        
    sc1x = multiplier*radius1/init_r1
    sc2x = multiplier*radius2/init_r2
    star1.scale = (sc1x, sc1x, sc1x)
    star2.scale = (sc2x, sc2x, sc2x)
    star1.material.color = hexcolor1
    star2.material.color = hexcolor2

    # See if we need to rescale the star sizes
    min(radius1, radius2)
    sc1x = multiplier*r1/init_r1
    sc2x = multiplier*r2/init_r2
    
    # Retrieve updated radial velocity or light curve based on the new orbital model
    if (fig_selector.value == rv_val):
        radvel_info = star.RadVelInfo(orbit_info, incl_slider.value)
        update_radvel_curve(radvel_info)
    else:
        light_curve = star.LightCurveInfo(orbit_info, incl_slider.value, radius1, radius2, temp1, temp2, Na, Ntheta)
        update_light_curve(light_curve)
        

        
def camera_update():
    '''
    Updated the camera for adjustments in gridsize since last projection
    '''
    global starcam, xmax, view_factor
    
    # Distance we are viewing from
    view_dist = view_factor*xmax
    
    # Define initial viewing position (adjusted for inclination)
    starcam.position = (view_dist*np.sin(incl_slider.value*star.deg2rad), 
                        0, 
                        view_dist*np.cos(incl_slider.value*star.deg2rad))
    
    # Define viewing region
    starcam.right=view_dist
    starcam.left=-view_dist 
    starcam.top=view_dist
    starcam.bottom=-view_dist
    starcam.far=2*view_dist

    # Tell the OrbitControl to update since I changed the position manually...
    controller.exec_three_obj_method('update',)

    return


def inclination_update(change):
    '''
    This function updates the star system's inclination to the plane of the sky.
    '''
    global xmax, incl_slider, starcam, controller, view_factor, xmax
    global orbit_info, radvel_info, light_curve, radius1, radius2, temp1, temp2
    
    # Current inclination value
    incl = change['new']*star.deg2rad
    
    # Define initial viewing position (adjusted for inclination)
    starcam.position = [view_factor*xmax*np.sin(incl), 
                        0, 
                        view_factor*xmax*np.cos(incl)]
    
    # Tell the OrbitControl to update since I changed the position manually...
    controller.exec_three_obj_method('update',)
    
    # Retrieve updated radial velocity or light curves and then update figure
    if (fig_selector.value == rv_val):
        radvel_info = star.RadVelInfo(orbit_info, incl_slider.value)
        update_radvel_curve(radvel_info)
    else:
        light_curve = star.LightCurveInfo(orbit_info, incl_slider.value, radius1, radius2, temp1, temp2, Na, Ntheta)
        update_light_curve(light_curve)
    
    
def position_update(change=None):
    '''
    This function updates the two stars' positions.
    '''
    global orbit_info, t_idx
    
    # Get previous orbital phase angle
    theta0 = np.arctan2(star1.position[1], star1.position[0])
    
    # Update the position
    t_idx = phase_slider.value
    star1.position = [orbit_info['x1_RSun'][t_idx], orbit_info['y1_RSun'][t_idx], 0]
    star2.position = [orbit_info['x2_RSun'][t_idx], orbit_info['y2_RSun'][t_idx], 0]

    # Get current orbital phase angle
    alpha = np.arctan2(star1.position[1], star1.position[0])
    dtheta = alpha - theta0
    
    # Rotate the stars (so they stay "tidally locked")
    star1.rotateZ(dtheta)
    star2.rotateZ(dtheta)
    
    # Update the phase line in radial velocity or light curve
    if (fig_selector.value == rv_val):
        rv_phase_line.x = [radvel_info['phase'][phase_slider.value], radvel_info['phase'][phase_slider.value]]
    else:
        lc_phase_line.x = [light_curve['phase'][phase_slider.value], light_curve['phase'][phase_slider.value]]


def graph_update(change=None):
    '''
    This function switches which graph to plot
    '''
    if (fig_selector.value == rv_val):
        radvel_info = star.RadVelInfo(orbit_info, incl_slider.value)
        new_fig = create_radvel_curve(radvel_info)
    else:
        light_curve = star.LightCurveInfo(orbit_info, incl_slider.value, radius1, radius2, temp1, temp2)
        new_fig = create_light_curve(light_curve)
    
    graph_fig.marks = new_fig.marks
    graph_fig.axes = new_fig.axes
    graph_fig.title = new_fig.title
    graph_fig.layout = new_fig.layout
    
    
def draw_orbits_RSun(orbit_info):
    """
    Take an orbit_info Pandas dataframe and return two Line objects representing the two stars orbits.
    """
    
    # Determine number of rows of orbit_info, set color of each segment
    N = len(orbit_info)
    orbitcolor = ['yellow']*N   # Changing the color doesn't seem to change line
    
    # Build Line object for star 1 orbit (to hover above plane)
    star1orbit = 5e-3*xmax*np.ones((N,3))
    star1orbit[:,0] = orbit_info['x1_RSun']
    star1orbit[:,1] = orbit_info['y1_RSun']
    vertices1 = star1orbit.tolist()
    orbit1_geom = p3j.Geometry(vertices=vertices1, colors=orbitcolor)
    orbit1_line = p3j.Line(geometry=orbit1_geom, material=p3j.LineBasicMaterial(linewidth=1, vertexColors='VertexColors'))
    
    # Build Line object for star 2 orbit (to hover above plane)
    star2orbit = 5e-3*xmax*np.ones((N,3))
    star2orbit[:,0] = orbit_info['x2_RSun']
    star2orbit[:,1] = orbit_info['y2_RSun']
    vertices2 = star2orbit.tolist()
    orbit2_geom = p3j.Geometry(vertices=vertices2, colors=orbitcolor)
    orbit2_line = p3j.Line(geometry=orbit2_geom, material=p3j.LineBasicMaterial(linewidth=1, vertexColors='VertexColors'))    
    
    return (orbit1_line, orbit2_line)


def create_light_curve(light_curve):
    '''
    Initialize the entire light curve 
    '''
    global lc_line, lc_phase_line
    
    # Set scales
    sc_x = bq.LinearScale()
    sc_y = bq.LinearScale()

    # Build the light curve
    lc_line = bq.Lines(x=light_curve['phase'], y=light_curve['F_norm'], scales={'x': sc_x, 'y': sc_y}, 
                      colors=['Black'])
    
    # Indicate the current phase
    x_phase = [light_curve['phase'][phase_slider.value], light_curve['phase'][phase_slider.value]]
    y_phase = [0, 1] 
    lc_phase_line = bq.Lines(x=x_phase, y=y_phase, scales={'x': sc_x, 'y': sc_y}, 
                      colors=['Red'])
    
    # Setup axes and return figure
    ax_x = bq.Axis(scale=sc_x, label='Phase')
    ax_y = bq.Axis(scale=sc_y, orientation='vertical', label='Fraction of Maximum Flux')
    return bq.Figure(marks=[lc_line, lc_phase_line], axes=[ax_x, ax_y], title='Light Curve',
                     layout=widgets.Layout(width=graph_width, height=graph_height))


def update_light_curve(light_curve):
    '''
    Update the light curve 
    '''
    global lc_line, lc_phase_line
    
    lc_line.x=light_curve['phase']
    lc_line.y=light_curve['F_norm']

    
def create_radvel_curve(radvel_info):
    '''
    Initialize the entire radial velocity curve
    '''
    global star1_line, star2_line, rv_phase_line
    
    # Set up the scales
    sc_x = bq.LinearScale()
    sc_y = bq.LinearScale()

    # Indicate the current phase
    x_phase = [radvel_info['phase'][phase_slider.value], radvel_info['phase'][phase_slider.value]]
    maxval = max( np.max(radvel_info['v1r']), np.max(radvel_info['v2r']) )
    minval = min( np.min(radvel_info['v1r']), np.min(radvel_info['v2r']) )
    y_phase = [minval, maxval] 
    rv_phase_line = bq.Lines(x=x_phase, y=y_phase, scales={'x': sc_x, 'y': sc_y}, 
                      colors=['Red'])
    
    # Draw the radial velocity curves
    star1_line = bq.Lines(x=radvel_info['phase'], y=radvel_info['v1r'], scales={'x': sc_x, 'y': sc_y},
                         colors=['DarkOrange'], labels=['Star 1'], display_legend=True)
    star2_line = bq.Lines(x=radvel_info['phase'], y=radvel_info['v2r'], scales={'x': sc_x, 'y': sc_y},
                         colors=['Blue'], labels=['Star 2'], display_legend=True)
    
    # Setup axes and return (initially invisible) figure
    ax_x = bq.Axis(scale=sc_x, label='Phase')
    ax_y = bq.Axis(scale=sc_y, orientation='vertical', label='Radial velocity (km/s)')
    ax_y.label_offset = '3.5em'
    return bq.Figure(marks=[star1_line, star2_line, rv_phase_line], axes=[ax_x, ax_y], title='Radial Velocity Curve',
                     layout=widgets.Layout(width=graph_width, height=graph_height))


def update_radvel_curve(radvel_info):
    '''
    Update the radial velocity curve 
    '''
    global star1_line, star2_line, rv_phase_line
    
    star1_line.x=radvel_info['phase']
    star1_line.y=radvel_info['v1r']
    star2_line.x=radvel_info['phase']
    star2_line.y=radvel_info['v2r']
    maxval = max( np.max(radvel_info['v1r']), np.max(radvel_info['v2r']) )
    minval = min( np.min(radvel_info['v1r']), np.min(radvel_info['v2r']) )
    rv_phase_line.y = [minval, maxval] 


def grid_setup(maxdist, update_surfgrid_pos=True):
    '''
    Set the grid separation based on the aphelion distance.
    '''
    order = pow(10,np.floor(np.log10(maxdist)))
    coeff = maxdist/order
    
    # Round up the maximum extent and then choose about 5 grid step
    xmax = np.ceil(coeff)*order
    grid_step = xmax/5
    
    # Move the surface grid up to avoid rendering issues
    if (update_surfgrid_pos):
        surfgrid.position = (0.0, 0.0, xmax/300)

    return(xmax, grid_step)


def OrbitalInfo_RSun(mass1, mass2, rad1, rad2, a, e, phi=0, N=1000):
    '''
    Computes the orbital information of interest in our prefered units and 
    also checks for possible stellar collisions.
    '''
    # Convert semimajor axis to expected input units
    a_AU = semimajor_slider.value*star.R_Sun/star.AU
        
    # Call orbital information routine (which works in AU and days)
    (P_days, ap, aa, maxrad, orbit_info) = star.OrbitalInfo(mass1, mass2, a_AU, e, phi, N)
    
    # Convert distances to solar radii
    ap *= star.AU/star.R_Sun
    aa *= star.AU/star.R_Sun
    maxrad *= star.AU/star.R_Sun
    
    orbit_info['r_RSun'] = orbit_info['r']*star.AU/star.R_Sun
    orbit_info['x1_RSun'] = orbit_info['x1']*star.AU/star.R_Sun
    orbit_info['y1_RSun'] = orbit_info['y1']*star.AU/star.R_Sun
    orbit_info['x2_RSun'] = orbit_info['x2']*star.AU/star.R_Sun
    orbit_info['y2_RSun'] = orbit_info['y2']*star.AU/star.R_Sun
    
    # Perform collision check
    if (ap < rad1+rad2):
        collision = True
    else:
        collision = False

    return (P_days, ap, aa, maxrad, orbit_info, collision)
    

In [ ]:
## INTERACTIVE/DISPLAY WIDGETS ##

# Define constants
min_mass = 0.2   # Maximum stellar mass in solar masses
max_mass = 24    # Maximum stellar mass in solar masses
mass_step = 0.1  # Step size for mass sliders in solar masses
init_mass = 1    # Initial mass of both stars in solar masses

min_a = 2         # Minimum semimajor axis of stars in solar radii
max_a = 400       # Maximum semimajor axis of stars in solar radii
step_a = 1        # Step size for separation slider in solar radii
init_a = 5        # Start off with the two stars close together

init_incl = 0.01  # Initial inclination value (non-zero value avoids odd orientation of FOV at start)
init_phi = 0      # Initial semimajor axis phase angle
init_ecc = 0.2    # Initial orbital eccentricity

view_factor = 1.25  # How many times the maximum distance to place the viewer
N = 1000          # Number of time steps to use for orbit
Na = 50           # Number of annuli to break up stars into for computing eclipse fraction
Ntheta = 180      # Number of angular steps to break up stars into for computing eclipse fraction

#
# Define some widths to use throughout for layout of controls
#

# Set simulation size
view_width = 450
view_height = view_width

# Initialize slider sizes
EntireWidth = '900px'
SimWidth = '{0:.0f}px'.format(view_width)
ControlColWidth = '425px'
slider_width = '300px'
slider_minwidth = '250px'
readout_width = '60px'
lum_width = '120px'
inform_width = '200px'
graph_width = '400px'
graph_height = '300px'

##
## Create control for selecting Figure
##
rv_val = 'Radial Velocity Curve'
lc_val = 'Light Curve'
fig_selector = widgets.RadioButtons(options=[rv_val, lc_val],
                                    value=rv_val,
                                    description='Plot to Display:',
                                    style = {'description_width': 'initial'},
                                    disabled=False,
                                    layout=widgets.Layout(width=slider_width, height='40px')
                                   )

##
##Create controls for star masses 
##

star1_slider = widgets.FloatSlider(
    value=init_mass,
    min=min_mass,
    max=max_mass+(mass_step/2),
    step=mass_step,
    description="Star 1 mass",
    style = {'description_width': 'initial'},
    disabled=False,
    continuous_update=False,
    orientation='horizontal',
    readout=False,
    readout_format='.1f',
    layout=widgets.Layout(width=slider_width, min_width=slider_minwidth)
)

star2_slider = widgets.FloatSlider(
    value=init_mass,
    min=min_mass,
    max=max_mass+(mass_step/2),
    step=mass_step,
    description="Star 2 mass",
    style = {'description_width': 'initial'},
    disabled=False,
    continuous_update=False,
    orientation='horizontal',
    readout=False,
    readout_format='.1f',
    layout=widgets.Layout(width=slider_width, min_width=slider_minwidth)
)

# Define text boxes for readout
star1_readout = widgets.BoundedFloatText(min=star1_slider.min, max=star1_slider.max, 
                                         value=star1_slider.value, layout=widgets.Layout(width=readout_width))
star2_readout = widgets.BoundedFloatText(min=star2_slider.min, max=star2_slider.max, 
                                         value=star2_slider.value, layout=widgets.Layout(width=readout_width))
# Link slider and textboxes
widgets.jslink((star1_readout, 'value'), (star1_slider, 'value'))
widgets.jslink((star2_readout, 'value'), (star2_slider, 'value'))

# Create the individual controls for stellar masses
solar_mass = widgets.Label('$M_\odot$')
star1_cntl = widgets.HBox([star1_slider, star1_readout, solar_mass], layout=widgets.Layout(width=ControlColWidth))
star2_cntl = widgets.HBox([star2_slider, star2_readout, solar_mass], layout=widgets.Layout(width=ControlColWidth))

##
##Create controls for system properties
##

# These sliders change entire orbital model and should NOT be continously updated
semimajor_slider = widgets.FloatSlider(
    value=init_a,
    min=min_a,
    max=max_a,
    step=step_a,
    description="Semimajor Axis",
    style = {'description_width': 'initial'},
    disabled=False,
    continuous_update=False,
    orientation='horizontal',
    readout=False,
    readout_format='.0f',
    layout=widgets.Layout(width=slider_width, min_width=slider_minwidth)
)

ecc_slider = widgets.FloatSlider(
    value=init_ecc,
    min=0,
    max=0.8,
    step=0.02,
    description="Eccentricity",
    style = {'description_width': 'initial'},
    disabled=False,
    continuous_update=False,
    orientation='horizontal',
    readout=False,
    readout_format='.2f',
    layout=widgets.Layout(width=slider_width, min_width=slider_minwidth)
)

phi_slider = widgets.FloatSlider(
    value=init_phi,
    min=0,
    max=180,
    step=1,
    description="Major Axis Longitude",
    style = {'description_width': 'initial'},
    disabled=False,
    continuous_update=False,
    orientation='horizontal',
    readout=False,
    readout_format='.0f',
    layout=widgets.Layout(width=slider_width, min_width=slider_minwidth)
)

incl_slider = widgets.FloatSlider(
    value=init_incl,
    min=0,
    max=90,
    step=1,
    description="Inclination",
    style = {'description_width': 'initial'},
    disabled=False,
    continuous_update=False,
    orientation='horizontal',
    readout=False,
    readout_format='.0f',
    layout=widgets.Layout(width=slider_width, min_width=slider_minwidth)
)

# Define text boxes for readout
semimajor_readout = widgets.BoundedFloatText(min=semimajor_slider.min, max=semimajor_slider.max, 
                                         value=semimajor_slider.value, layout=widgets.Layout(width=readout_width))
ecc_readout = widgets.BoundedFloatText(min=ecc_slider.min, max=ecc_slider.max, 
                                         value=ecc_slider.value, layout=widgets.Layout(width=readout_width))
incl_readout = widgets.BoundedFloatText(min=incl_slider.min, max=incl_slider.max, 
                                         value=incl_slider.value, layout=widgets.Layout(width=readout_width))
phi_readout = widgets.BoundedFloatText(min=phi_slider.min, max=phi_slider.max, 
                                         value=phi_slider.value, layout=widgets.Layout(width=readout_width))
# Link slider and textboxes
widgets.jslink((semimajor_readout, 'value'), (semimajor_slider, 'value'))
widgets.jslink((ecc_readout, 'value'), (ecc_slider, 'value'))
widgets.jslink((incl_readout, 'value'), (incl_slider, 'value'))
widgets.jslink((phi_readout, 'value'), (phi_slider, 'value'))

# Create the individual controls for system properties
Solar_radius = widgets.Label('$R_\odot$')
deg_label = widgets.HTML('&deg;')
semimajor_cntl = widgets.HBox([semimajor_slider, semimajor_readout, Solar_radius], layout=widgets.Layout(width=ControlColWidth))
ecc_cntl = widgets.HBox([ecc_slider, ecc_readout], layout=widgets.Layout(width=ControlColWidth))
incl_cntl = widgets.HBox([incl_slider, incl_readout, deg_label], layout=widgets.Layout(width=ControlColWidth))
phi_cntl = widgets.HBox([phi_slider, phi_readout, deg_label], layout=widgets.Layout(width=ControlColWidth))

##
## Orbital playback controls
##
phase_slider = widgets.FloatSlider(
    value=0,
    min=0,
    max=N,
    step=1,
    description="Phase",
    style = {'description_width': 'initial'},
    disabled=False,
    continuous_update=True,
    orientation='horizontal',
    readout=False,
    readout_format='.0f',
    layout=widgets.Layout(width=slider_width, min_width=slider_minwidth)
)
phase_play = widgets.Play(interval = 1, value = phase_slider.min, min=phase_slider.min, max=phase_slider.max, 
                          step=1, description="Press play", disabled=False, _repeat=True, show_repeat=False)
widgets.jslink((phase_play, 'value'), (phase_slider, 'value'))
phase_cntl = widgets.HBox([phase_slider, phase_play], layout=widgets.Layout(width=ControlColWidth))

##
## Create text boxes for reporting certain system parameters
##
P_output = widgets.Text(
    value = str(0),
    description = 'Orbital Period (Days)',
    style = {'description_width': 'initial'},
    disabled = True, 
    layout=widgets.Layout(width=inform_width)  
)

ap_output = widgets.Text(
    value = str(0),
    description = 'Periastron ($R_\odot$)',
    style = {'description_width': 'initial'},
    disabled = True, 
    layout=widgets.Layout(width=inform_width)
)

aa_output = widgets.Text(
    value = str(0),
    description = 'Apastron ($R_\odot$)',
    style = {'description_width': 'initial'},
    disabled = True, 
    layout=widgets.Layout(width=inform_width)  
)

gridsep_output = widgets.Text(
    value = str(0),
    description = 'Grid Spacing ($R_\odot$)',
    style = {'description_width': 'initial'},
    disabled = True, 
    layout=widgets.Layout(width=inform_width)  
)

luminosity = widgets.Label('Luminosity ($L_\odot$)')

L1_output = widgets.Text(
    value = str(0),
    description = 'Star 1: ',
    style = {'description_width': 'initial'},
    disabled = True, 
    layout=widgets.Layout(width=lum_width)  
)

L2_output = widgets.Text(
    value = str(0),
    description = 'Star 2: ',
    style = {'description_width': 'initial'},
    disabled = True, 
    layout=widgets.Layout(width=lum_width)  
)

lum_info = widgets.HBox([luminosity, L1_output, L2_output])

In [ ]:
# Initialize orbit control sliders
semimajor_slider.value = init_a
ecc_slider.value = init_ecc
incl_slider.value = init_incl
phi_slider.value = init_phi

# Initialize stellar mass sliders
star1_slider.value = init_mass
star2_slider.value = init_mass

# Set initial parameters based on stellar mass assuming main sequence stars
(radius1, temp1, hexcolor1, radius2, temp2, hexcolor2) = ConfigBothStars(star1_slider.value, star2_slider.value)
r1 = radius1
r2 = radius2
multiplier = 1   # Use this to increase the apparent size of the stars if they get too small

# Set luminosities
L1_output.value = str(nf.SigFig((temp1/star.Te_Sun)**4 * radius1**2, 3))
L2_output.value = str(nf.SigFig((temp2/star.Te_Sun)**4 * radius2**2, 3))

# Save initial radius to scale all other radii to this
init_r1 = r1   # global variables for initial radii
init_r2 = r2   # global variables for initial radii
sc1x = multiplier*r1/init_r1
sc2x = multiplier*r2/init_r2
scale1 = (sc1x, sc1x, sc1x)
scale2 = (sc2x, sc2x, sc2x)

# Initialize time index
t_idx = 0

# Determine initial orbital parameters
a_AU = semimajor_slider.value*star.R_Sun/star.AU
(P_days, ap, aa, maxrad, orbit_info, collision) = OrbitalInfo_RSun(star1_slider.value, star2_slider.value,
                                                                   radius1, radius2,
                                                                   semimajor_slider.value, 
                                                                   ecc_slider.value,
                                                                   phi_slider.value, N)
# Compute radial velocity and light curves
radvel_info = star.RadVelInfo(orbit_info, incl_slider.value)
light_curve = star.LightCurveInfo(orbit_info, incl_slider.value, radius1, radius2, temp1, temp2, Na, Ntheta)

# Convert units
P_output.value = "{0:.1f}".format(P_days)
ap_output.value = "{0:.1f}".format(ap)
aa_output.value = "{0:.1f}".format(aa)

##
## Set Up 3D Simulation and controls for left side
##

# Create stars at the appropriate positions with appropriate characteristics
star1 = star.StarMesh(temp1, r1, scale1, [orbit_info['x1_RSun'][t_idx], orbit_info['y1_RSun'][t_idx], 0])
star2 = star.StarMesh(temp2, r2, scale2, [orbit_info['x2_RSun'][t_idx], orbit_info['y2_RSun'][t_idx], 0])

# Initialize a flat surface to contain orbital plane (accounting for size of orbital plane + star radius)
(xmax, grid_step) = grid_setup(maxrad + max(r1,r2), update_surfgrid_pos=False)
gridsep_output.value = "{0:.2f}".format(grid_step)
surf, surfgrid = star.xyplane(xmax, grid_step)  # Generate flat surface and grid for perspective
view_dist = view_factor*xmax

# Draw their orbits
(orbit1_line, orbit2_line) = draw_orbits_RSun(orbit_info)

# Makes the scene environment
scene2 = p3j.Scene(children=[star1, star2, surfgrid, surf, orbit1_line, orbit2_line], background='black')

# Define initial viewing position (only needed for Perspective Camera
init_position = (view_dist*np.sin(incl_slider.value*star.deg2rad), 
                 0, 
                 view_dist*np.cos(incl_slider.value*star.deg2rad))

# Creates the camera so you can see stuff (with z-axis oriented up as consistent with inclination to line of sight)
#starcam = p3j.PerspectiveCamera(position=init_position, up=[0, 0, view_factor*xmax])
# Using OrthographicCamera instead of PerspectiveCamera to show view from Earth (far away) where both stars are really
# at the same distance.
#
# NOTE: No need for children=[p3j.DirectionalLight(color='white', position=[1.5*xmax, 1.5*xmax, 1.5*xmax], intensity=1)] 
# since I am using BasicMeshs which don't take into account external lighting.                      
starcam = p3j.OrthographicCamera(right=view_dist, left=-view_dist, 
                                 top=view_dist, bottom=-view_dist,
                                 near=0, far=2*view_dist, 
                                 position=init_position, 
                                 up=[0, 0, view_dist])

# Makes a controller for the starcam camera looking toward the origin
controller = p3j.OrbitControls(controlling=starcam, autoRotate=True, enableRotate=False, enableZoom=False, 
                               target = [0, 0, 0])

# creates the object that gets displayed to the screen
renderer2 = p3j.Renderer(camera=starcam, 
                    scene=scene2, 
                    controls=[controller],
                    width=view_width, height=view_height)

# Create play button to control phase value automatically
phase_title_default = '<b>Controls for Orbital Motion</b>:'
phase_title_collision = '<B style="color:red">COLLISION DETECTED! CONTROLS DISABLED!</B>'
phase_title = widgets.HTML(phase_title_default)
phase_controls = widgets.VBox([phase_title, phase_cntl], layout=widgets.Layout(width=SimWidth))

# Assemble items in left column
left_column = widgets.VBox([widgets.HTML('<h3>Model View</h3>'), renderer2, gridsep_output, phase_controls])


##
## Build Controls on right side
##

# Compute light curve and radial velocoity curve and select which to display initially
if (fig_selector.value == rv_val):
    graph_fig = create_radvel_curve(radvel_info)
else:
    graph_fig = create_light_curve(light_curve)

# Spacer widget
spacer = widgets.HTML('<p>', layout=widgets.Layout(width='10px'))

# Creates Stellar Mass Controls
star_title = widgets.HTML('<b>Stellar Properties</b>:')
star_controls = widgets.VBox([star_title, star1_cntl, star2_cntl, widgets.HBox([spacer, lum_info])])
              
# Creates System Parameter Controls (e.g. orbital property controls)
sys_title_default = '<b>System Parameters</b>:'
sys_title_scaling = '<b>System Parameters</b> (<B style="color:red">Stars NOT TO SCALE!</B>):'
sys_title = widgets.HTML(value=sys_title_default)
starorbit_controls = widgets.VBox([sys_title, 
                                   semimajor_cntl,
                                   widgets.HBox([spacer, widgets.VBox([P_output])]),
                                   ecc_cntl,  
                                   incl_cntl, 
                                   phi_cntl, 
                                   spacer])


# Creates a vertical box for the right control panel
right_column = widgets.VBox([graph_fig, fig_selector, star_controls, starorbit_controls], layout=widgets.Layout(width=ControlColWidth) )

# Places the figure, sliders, and output into a Vbox. The figure is 
# alone in the top, while the sliders and output are in a Hbox
# inside the bottom of the Vbox.
MainDisplay = widgets.HBox([left_column, spacer, right_column])

# Sets the dimensions of the box. Sets the entire width and the height of 
# just the top.
MainDisplay.layout.width = EntireWidth
MainDisplay.layout.overflow_x = 'hidden'
display(MainDisplay)

##
## Turn on interactivity by linking sliders to functions changing the simulation.
##

# This control determines which graph to display
fig_selector.observe(graph_update, names=['value'])

# These sliders change the orbital simulation properties
star1_slider.observe(property_update, names=['value'])
star2_slider.observe(property_update, names=['value'])
semimajor_slider.observe(property_update, names=['value'])
ecc_slider.observe(property_update, names=['value'])
phi_slider.observe(property_update, names=['value'])

# This slider just changes our viewpoint (and radial velocities)
incl_slider.observe(inclination_update, names=['value'])

# This slider just changes what phase of the orbit to display
phase_slider.observe(position_update, names=['value'])